In [3]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

In [10]:
# parameters
customer_id = '134'
formatted_attribute = 'variable_speed_belt_overall_height'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

bucket_name = """Variable Speed Belt"""

# regex_pattern = r"(?i)([^\.]\d* in\w* Belt Width)|(\d*\.\d* in\w* Belt Width)|(\d*\-\d*\/\d* in\w* Belt Width)|(\d*\.\d in\w* Belt Width)"
regex_pattern = r"(?i)([^\.]\d* in Belt Width)|(\d*\.\d* in Belt Width)"


value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'bucket_name': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [11]:
metric = ' Inches'

range_string = """
 0 - 1 Inches
 1.1 - 2 Inches
 2.1 - 3 Inches
 3.1 - 4 Inches
 4.1 - 5 Inches
 Greater than 5 Inches
"""

In [12]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [13]:
df = query_from_file(file_name='query/end_all_be_all.sql', params=params)
df

attribute            value      customers_name  \
0    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
1    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
2    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
3    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
4    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
..                                  ...              ...                 ...   
813  variable_speed_belt_overall_height              n/a  motionindustriesus   
814  variable_speed_belt_overall_height              n/a  motionindustriesus   
815  variable_speed_belt_overall_height              n/a  motionindustriesus   
816  variable_speed_belt_overall_height              n/a  motionindustriesus   
817  variable_speed_belt_overall_height              n/a  motionindustriesus   

    external_id  \
0      00690366   
1      00690887   
2      00690958   
3      00691028   
4      00691182   
..          ...   
813    08837007   
814    08837008   
815    08837009   
816    08837010   
817    08837011   

                                                                                                                  long_desc  \
0     Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   
1       Multi-Speed® 9804-0479 Multi-Speed Belt - 1-7/16 in Top Width, 38 in Overall Length, 0.40 in Overall Height, Rubber   
2    Multi-Speed® 9800-0069 Multi-Speed Belt - 1.569 in Top Width, 30.65 in Overall Length, 0.318 in Overall Height, Rubber   
3    Multi-Speed® 9800-0260 Multi-Speed Belt - 1.579 in Top Width, 62.49 in Overall Length, 0.467 in Overall Height, Rubber   
4    Multi-Speed® 9800-0904 Multi-Speed Belt - 1.563 in Top Width, 149.74 in Overall Length, 0.47 in Overall Height, Rubber   
..                                                                                                                      ...   
813                                     PowerLink® 9808-1602 Variable Speed Belt - 18 in Top Width, 30.75 in Overall Length   
814                                     PowerLink® 9808-1705 Variable Speed Belt - 20 in Top Width, 32.87 in Overall Length   
815                                     PowerLink® 9808-1707 Variable Speed Belt - 20 in Top Width, 29.25 in Overall Length   
816                                     PowerLink® 9808-1708 Variable Speed Belt - 20 in Top Width, 33.15 in Overall Length   
817                                     PowerLink® 9808-1709 Variable Speed Belt - 20 in Top Width, 33.31 in Overall Length   

    resolution          bucket_name  
0        rules  Variable Speed Belt  
1        rules  Variable Speed Belt  
2        rules  Variable Speed Belt  
3        rules  Variable Speed Belt  
4        rules  Variable Speed Belt  
..         ...                  ...  
813       bulk  Variable Speed Belt  
814       bulk  Variable Speed Belt  
815       bulk  Variable Speed Belt  
816       bulk  Variable Speed Belt  
817       bulk  Variable Speed Belt  

[818 rows x 7 columns]

In [13]:
# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0        []
1        []
2        []
3        []
4        []
         ..
13302    []
13303    []
13304    []
13305    []
13306    []
Name: long_desc, Length: 13307, dtype: object

In [14]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [15]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

      external_id  \
0        00710495   
1        00710497   
2        00710498   
3        00710500   
4        00710501   
...           ...   
13302    00713646   
13303    00713908   
13304    00711347   
13305    00711427   
13306    00711433   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution                 matches  
0       standard  [0.3750 in Belt Width]  
1       standard  [0.3750 in Belt Width]  
2       standard  [0.3750 in Belt Width]  
3       standard  [0.3750 in Belt Width]  
4       standard  [0.3750 in Belt Width]  
...          ...                     ...  
13302       bulk  [1.5000 in Belt Width]  
13303       bulk  [3.0000 in Belt Width]  
13304       bulk  [0.7500 in Belt Width]  
13305       bulk  [1.0000 in Belt Width]  
13306       bulk  [1.0000 in Belt Width]  

[13307 rows x 7 columns]

In [16]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()
df

attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

      external_id  \
0        00710495   
1        00710497   
2        00710498   
3        00710500   
4        00710501   
...           ...   
13302    00713646   
13303    00713908   
13304    00711347   
13305    00711427   
13306    00711433   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution matches  
0       standard      []  
1       standard      []  
2       standard      []  
3       standard      []  
4       standard      []  
...          ...     ...  
13302       bulk      []  
13303       bulk      []  
13304       bulk      []  
13305       bulk      []  
13306       bulk      []  

[13307 rows x 7 columns]

In [19]:
df[df['matches'].astype(str) != '[]']


Empty DataFrame
Columns: [attribute, value, customer_name, external_id, long_desc, resolution, matches]
Index: []

In [18]:
matches = df[df['matches'].astype(str) != '[]'] #Find all non blank matches
matches

Empty DataFrame
Columns: [attribute, value, customer_name, external_id, long_desc, resolution, matches]
Index: []

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
matches = df[df['matches'].astype(str) != '[]'] 
matches 

In [ ]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [ ]:
matches

In [ ]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))
#matches['matches'].replace(r'\-','',regex=True,inplace=True)
matches['matches'].value_counts()

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))

In [20]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 1:
        return " 0 - 1 Inches"
        
    if num >=  1.1 and num <= 2:
        return " 1.1 - 2 Inches"
        
    if num >=  2.1 and num <= 3:
        return " 2.1 - 3 Inches"
        
    if num >=  3.1 and num <= 4:
        return " 3.1 - 4 Inches"
        
    if num >=  4.1 and num <= 5:
        return " 4.1 - 5 Inches"
        
    if num >=  5:
        return " Greater than 5 Inches"
        
    else:
        return r"n/a"



In [ ]:
matches[curation_col] = matches['matches'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Motion Industries', attribute, df, matches)